In [1]:
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import torch

In [9]:
import numpy as np

In [3]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the training and test datasets
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainset_reduzido = torch.utils.data.random_split(trainset, [1000, len(trainset) - 1000])[0]
# Create data loaders
trainloader = DataLoader(trainset_reduzido, batch_size=256, shuffle=True)

In [33]:
net = Net()
params_init = [param.clone().detach() for param in net.parameters()]
for i, param in enumerate(net.parameters()):
    if i == 9:
        print(f'parametro inicial, camada final: {param[0]}')
grads_acu = [torch.zeros_like(param) for param in net.parameters()]
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
net.train()
for epoch in range(2):
    for data in trainloader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        params_anterior = [param.clone().detach() for param in net.parameters()]
        loss.backward()
        optimizer.step()
        for i, param in enumerate(net.parameters()):
            grads_acu[i] += param.grad
            # if i == 9:
            #     print(f'parametro anterior: {params_anterior[i][0]}')
            #     print(f'gradiente da iteracao: {param.grad[0]}')
            #     print(f'parametro anterior - gradiente: {params_anterior[i][0] - 0.01*param.grad[0]}')
            #     print(f'parametro atualizado: {param[0]}')
gradients = [grad.cpu().numpy() for grad in grads_acu] 
for i, param in enumerate(net.parameters()):
    if i == 9:
        print(f'parametros finais, camada final: {param[0]} gradi total: {gradients[9][0]}')

parametro inicial, camada final: 0.04712291806936264
parametros finais, camada final: 0.046314749866724014 gradi total: 0.08081717044115067


In [26]:
y = np.random.randint(1, 12000, size=10)

In [31]:
y

array([ 7084,   288,  2117,  4414,   567, 10940,  8444,  6988,  3009,
        4693])

In [21]:
def _try_inplace(x, y, np_binary_op):
        return (  # type: ignore[no-any-return]
            np_binary_op(x, y, out=x)
            if np.can_cast(y, x.dtype, casting="same_kind")
            else np_binary_op(x, np.array(y, x.dtype), out=x)
        )

In [35]:
params = [
        _try_inplace(x, y[0], np_binary_op=np.multiply)
        for x in gradients
    ]

In [39]:
import numpy as np
from functools import partial, reduce

def _try_inplace(x, y, np_binary_op):
    return np_binary_op(x, y, out=x)

# Parâmetros e gradientes de exemplo
params = [np.array([1, 2, 3]), np.array([4, 5, 6])]
res = [np.array([1, 2, 3]), np.array([4, 5, 6])]

# Atualização dos parâmetros
params = [
    reduce(partial(_try_inplace, np_binary_op=np.add), layer_updates)
    for layer_updates in zip(params, res)
]

print(params)

[array([2, 4, 6]), array([ 8, 10, 12])]


In [55]:
grad_ex = [0.01,0.02,0.3,0.04,0.05,0.06,0.07,0.08,0.09,0.010]
scale = [0.1]*10
scale = grad_ex/np.sum(grad_ex)
np.sum(np.multiply(grad_ex, scale))

0.16123287671232875